In [0]:
#!pip install chainladder

In [0]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.sql.types import LongType
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# Creating Spark Object

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
#dbutils.fs.rm("/FileStore/df/reinsurance_premiums3.csv/_SUCCESS")
#dbutils.fs.rm("/FileStore/df/reinsurance_premiums3.csv/_committed_4267799952902735037")
#dbutils.fs.rm("/FileStore/df/reinsurance_premiums3.csv/_started_4267799952902735037")
#dbutils.fs.rm("/FileStore/df/reinsurance_premiums3.csv/part-00000-tid-4267799952902735037-377f785d-cb32-4bbe-a255-fbd569d78d14-28-1-c000.csv.gz")
#dbutils.fs.rm("/FileStore/df/reinsurance_premiums3.csv")

In [0]:
# Schema Interger
def schema_integer(df):
  cols=df.columns
  for i in cols:
    df=df.withColumn(i,col(i).cast(IntegerType()))
  return df

In [0]:
mortgage_data=spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/srivatsramaswamy10@gmail.com/Reinsurance.csv")
#mortgage_data=mortgage_data.where(mortgage_data['Category']=='Premium')
#mortgage_data=mortgage_data.where(mortgage_data['Class of business category']=='Inwards reinsurance')
#mortgage_data=mortgage_data.where(mortgage_data['subject']=='Gross')
#mortgage_data=mortgage_data.where(mortgage_data['Data Item']=='Gross case estimates by class of business')
mortgage_data.show()

+--------------------+-------+--------------------+--------------------+-----------------------+--------------------------+-------------------+--------------------+--------------------------+----------------+-------------+
|Reporting period end|Subject|            Category|           Data Item|Class of business group|Class of business category|  Class of business|Reporting year basis|Accident/underwriting year|Development year|        Value|
+--------------------+-------+--------------------+--------------------+-----------------------+--------------------------+-------------------+--------------------+--------------------------+----------------+-------------+
|          31/12/2020|  Gross|Ultimate cost com...|Gross case estima...|    Inwards reinsurance|       Inwards reinsurance|Inwards reinsurance|   Underwriting year|                      2020|               0|1155000000.00|
|          31/12/2020|  Gross|Ultimate cost com...|Gross case estima...|    Inwards reinsurance|       Inwar

In [0]:
mortgage=mortgage_data.select('Accident/underwriting year','Development year','Value')
#mortgage=mortgage.filter(mortgage['Development year']!=11)
#mortgage=mortgage.filter(mortgage['Development year']!=10)
#mortgage=mortgage.filter(mortgage['Development year']!=9)
#mortgage=mortgage.filter(mortgage['Accident/underwriting year']!=2002)
#mortgage=mortgage.filter(mortgage['Accident/underwriting year']!=2003)
#mortgage=mortgage.filter(mortgage['Accident/underwriting year']!=2004)
#mortgage=mortgage.filter(mortgage['Accident/underwriting year']!=2005)
#mortgage=mortgage.filter(mortgage['Accident/underwriting year']!=2006)
#mortgage=mortgage.filter(mortgage['Accident/underwriting year']!=2007)
#mortgage=mortgage.filter(mortgage['Accident/underwriting year']!=2008)
#mortgage=mortgage.filter(mortgage['Accident/underwriting year']!=2009)
#mortgage=mortgage.filter(mortgage['Accident/underwriting year']!=2010)
#mortgage=mortgage.filter(mortgage['Accident/underwriting year']!=2011)
#mortgage=mortgage.withColumn('Value',mortgage['Value']/1000000)
mortgage=mortgage.withColumn('Accident/underwriting year',col('Accident/underwriting year').cast(IntegerType()))
mortgage=mortgage.withColumn('Development year',col('Development year').cast(IntegerType()))
mortgage=mortgage.withColumn('Value',col('Value').cast(LongType()))
mortgage=mortgage.dropna()
mortgage.show()

+--------------------------+----------------+----------+
|Accident/underwriting year|Development year|     Value|
+--------------------------+----------------+----------+
|                      2020|               0|1155000000|
|                      2019|               1| 782000000|
|                      2018|               2| 526000000|
|                      2017|               3| 425000000|
|                      2016|               4| 401000000|
|                      2015|               5| 275000000|
|                      2014|               6| 181000000|
|                      2013|               7|  75000000|
|                      2012|               8|  51000000|
|                      2011|               9| 138000000|
|                      2010|              10| 172000000|
|                      2020|               0|2646000000|
|                      2019|               1|4059000000|
|                      2018|               2|3507000000|
|                      2017|   

In [0]:
mortgage=mortgage.groupBy("Accident/underwriting year","Development year").sum("Value")
mortgage=mortgage.orderBy(mortgage['Accident/underwriting year'].asc(),mortgage['Development year'].asc())
mortgage.show()

+--------------------------+----------------+----------+
|Accident/underwriting year|Development year|sum(Value)|
+--------------------------+----------------+----------+
|                      2002|              10|1714000000|
|                      2003|               9|1554000000|
|                      2003|              10|1532000000|
|                      2004|               8|1613000000|
|                      2004|               9|1623000000|
|                      2004|              10|1631000000|
|                      2005|               7|1752000000|
|                      2005|               8|1758000000|
|                      2005|               9|1755000000|
|                      2005|              10|  17000000|
|                      2006|               6|1699000000|
|                      2006|               7|1719000000|
|                      2006|               8|1714000000|
|                      2006|               9|1708000000|
|                      2006|   

In [0]:
mortgage_bootstraped=mortgage.sample(withReplacement=True,fraction=100000.0)
mortgage_bootstraped.show(100)

+--------------------------+----------------+----------+
|Accident/underwriting year|Development year|sum(Value)|
+--------------------------+----------------+----------+
|                      2002|              10|1714000000|
|                      2002|              10|1714000000|
|                      2002|              10|1714000000|
|                      2002|              10|1714000000|
|                      2002|              10|1714000000|
|                      2002|              10|1714000000|
|                      2002|              10|1714000000|
|                      2002|              10|1714000000|
|                      2002|              10|1714000000|
|                      2002|              10|1714000000|
|                      2002|              10|1714000000|
|                      2002|              10|1714000000|
|                      2002|              10|1714000000|
|                      2002|              10|1714000000|
|                      2002|   

In [0]:
mortgage_final=mortgage_bootstraped.groupBy("Accident/underwriting year","Development year").sum("sum(Value)") #.sum('Value')
mortgage_final.show()

+--------------------------+----------------+---------------+
|Accident/underwriting year|Development year|sum(sum(Value))|
+--------------------------+----------------+---------------+
|                      2002|              10|171562830000000|
|                      2003|               9|154177002000000|
|                      2003|              10|153177020000000|
|                      2004|               8|160777388000000|
|                      2004|               9|161886135000000|
|                      2004|              10|163282672000000|
|                      2005|               7|175471560000000|
|                      2005|               8|175719132000000|
|                      2005|               9|175387680000000|
|                      2005|              10|  1698419000000|
|                      2006|               6|169877913000000|
|                      2006|               7|172252395000000|
|                      2006|               8|171461704000000|
|       

In [0]:
# Finally creating the development year in a proper year format for generating a development triangle

mortgage_final=mortgage_final.withColumn("Development year",mortgage_final["Accident/underwriting year"] + mortgage_final["Development year"])
mortgage_final.show()

+--------------------------+----------------+---------------+
|Accident/underwriting year|Development year|sum(sum(Value))|
+--------------------------+----------------+---------------+
|                      2002|            2012|171562830000000|
|                      2003|            2012|154177002000000|
|                      2003|            2013|153177020000000|
|                      2004|            2012|160777388000000|
|                      2004|            2013|161886135000000|
|                      2004|            2014|163282672000000|
|                      2005|            2012|175471560000000|
|                      2005|            2013|175719132000000|
|                      2005|            2014|175387680000000|
|                      2005|            2015|  1698419000000|
|                      2006|            2012|169877913000000|
|                      2006|            2013|172252395000000|
|                      2006|            2014|171461704000000|
|       

In [0]:
#mortgage_final=mortgage_final.withColumnRenamed('sum(Value)','Amount')
mortgage_final=mortgage_final.withColumnRenamed('sum(sum(Value))','Amount')
mortgage_final.show()

+--------------------------+----------------+---------------+
|Accident/underwriting year|Development year|         Amount|
+--------------------------+----------------+---------------+
|                      2002|            2012|171562830000000|
|                      2003|            2012|154177002000000|
|                      2003|            2013|153177020000000|
|                      2004|            2012|160777388000000|
|                      2004|            2013|161886135000000|
|                      2004|            2014|163282672000000|
|                      2005|            2012|175471560000000|
|                      2005|            2013|175719132000000|
|                      2005|            2014|175387680000000|
|                      2005|            2015|  1698419000000|
|                      2006|            2012|169877913000000|
|                      2006|            2013|172252395000000|
|                      2006|            2014|171461704000000|
|       

In [0]:
#mortgage_final=mortgage_final.withColumn("Development year",mortgage_final["Accident/underwriting year"] + mortgage_final["Development year"])
#mortgage_final=mortgage_final.withColumnRenamed('sum(Value)','Amount')
#mortgage_final.show()

In [0]:
#mortgage=mortgage.withColumn("Development year",mortgage["Accident/underwriting year"] + mortgage["Development year"])
#mortgage=mortgage.withColumnRenamed('sum(Value)','Amount')
#mortgage.show()

In [0]:
mortgage_final.coalesce(1).write.format("com.databricks.spark.csv").option("header","true").option("compression","gzip").save("dbfs:/FileStore/df/reinsurance_claims_new.csv")

In [0]:
#mortgage.coalesce(1).write.format("com.databricks.spark.csv").option("header","true").option("compression","gzip").save("dbfs:/FileStore/df/reinsurance_case.csv")

In [0]:
#mortgage.coalesce(1).write.format("com.databricks.spark.csv").option("header","true").option("compression","gzip").save("dbfs:/FileStore/df/reinsurance_premiums3.csv")

In [0]:
input_path='dbfs:/FileStore/df/reinsurance_claims_new.csv'
#input_path='dbfs:/FileStore/df/reinsurance_premiums3.csv'
in_data = spark.read.csv(input_path,header=True)

In [0]:
#in_data=in_data.withColumnRenamed('Value','Amount')

In [0]:
#in_data=in_data.withColumn('Amount',in_data['Amount'])
#in_data.show()

In [0]:
display(in_data)

Accident/underwriting year,Development year,Amount
2002,2012,171562830000000
2003,2012,154177002000000
2003,2013,153177020000000
2004,2012,160777388000000
2004,2013,161886135000000
2004,2014,163282672000000
2005,2012,175471560000000
2005,2013,175719132000000
2005,2014,175387680000000
2005,2015,1698419000000
